# SBM2 configuration tester

This notebook runs the SBM2 detection algorithm with a selected configuration and prints a summary. The summarry includes:
- dates and hours on which the algoritm triggered
    - dates where the algorithm made a correct detection
    - dates where the algorithm made an incorrect detection
    - unknown dates which can be added (counted as incorrect detections by default)
- configuration accuracy
- more?

This notebook can be run and edited via a mybinder link in an online environment. 

Run the cells below and in the ```# Evaluating``` cell, choose which configuration you want to test.

In [2]:
# Import cell
import numpy as np
import pickle

In [26]:
# Functions for getting number of events in rolling buffer (DT_LW)
def AMP_Check(AMP, EPEAK_THRESH): 
    return AMP>EPEAK_THRESH

def EVENTS_Check(NR, NW_THRES): 
    return NR>NW_THRES

def FREQ_Check(Fp, FP_MIN, FP_MAX, RUA): 
    fmin = Fp>(FP_MIN/RUA)
    fmax = Fp<(FP_MAX/RUA)
    temp = fmin.astype(int)+fmax.astype(int)
    temp[temp<2] = 0
    temp[temp==2] = 1
    return temp

def SBM2_Check(AMP, NR, FP, EPEAK_THRESH, NW_THRES, FP_MIN, FP_MAX, RUA):
    A = AMP_Check(AMP, EPEAK_THRESH)
    E = EVENTS_Check(NR, NW_THRES)
    F = FREQ_Check(FP, FP_MIN, FP_MAX, RUA)
    A = A.astype(int)
    E = E.astype(int)
    F = F.astype(int)
    temp = A + E + F
    condmet = A + E + F
    temp[temp<3] = 0
    temp[temp==3] = 1
    temp = temp.astype(int)
    return temp, condmet

def SBM2_Find0(EV, Epoch, DT):
    counter = np.zeros(EV.size)
    for i in  zip(*np.where(EV>0)):
        ind = i[0]
        t1 = Epoch[ind]
        t0 = t1 - DT*1e9
        EP0 = Epoch[0:ind]
        counter[ind] = np.sum(EV[np.where(EP0>t0)])
    return counter

def SBM2_Find(ev, epoch, dt):
    counter = np.zeros(ev.size)
    temp = np.where(ev>0)
    for i in temp[0].tolist():
        t0 = epoch[i]
        t1 = t0 + dt*1e9
        counter[(epoch>=t0) & (epoch<t1)] += 1
    return counter

def get_triggers(EIRB, TH, Ep,time, delay):
    Ep =Ep*1e-9
    x = Ep[np.where(EIRB>TH)]
    tEp = np.array([])
    for i in x:
        if i==x[0]:
            t0 = i
            tEp = np.append(tEp, t0)
        else:
            if i-delay>t0:
                t0 = i
                tEp = np.append(tEp, t0)
    indxs = np.array([])
    for i in tEp:
        ind = np.where(Ep == i)
        indxs = np.append(indxs, ind)
    return indxs

def set_conf(conf):
    EPEAK_THRES = conf['EPEAK_THRES']
    FP_MIN = conf['FP_MIN'] 
    FP_MAX = conf['FP_MAX'] 
    NW_THRES = conf['NW_THRES']
    DT_LW = conf['DT_LW']
    N_LW_THRES = conf['N_LW_THRES']
    DT_SBM2 = conf['DT_SBM2']
    RUA = conf['RUA']
    return EPEAK_THRES, FP_MIN, FP_MAX, NW_THRES, DT_LW, N_LW_THRES, DT_SBM2, RUA

def evaluate(conf, loudness=1):
    EPEAK_THRES, FP_MIN, FP_MAX, NW_THRES, DT_LW, N_LW_THRES, DT_SBM2, RUA = set_conf(conf)
    
    EV, condmet = SBM2_Check(AMP, NR, FP, EPEAK_THRES, NW_THRES, FP_MIN, FP_MAX, rua)
    EIRB = SBM2_Find(EV, Epoch, DT_LW)
    triggerinx = get_triggers(EIRB, N_LW_THRES, Epoch,time, delay = DT_SBM2)
    
    Ep = Epoch*1e-9
    times = np.array([])
    for i in list(triggerinx.astype(int).tolist()):
        times = np.append(times, time[i])
    
    # Known dates - sbm includes known type III. events; fp includes false positives often returned by SBM2 detection algorithm.
    # Check dates and add them to the correct list if the Evaluation cell printed 'check DATE'. 
    sbm = ['2022/01/18','2021/12/04','2021/12/05','2021/12/06','2021/09/28','2021/09/27','2021/09/26','2021/09/24','2021/07/24','2021/08/24','2021/10/06','2021/10/28','2021/10/09','2021/08/26','2021/05/22','2021/05/23','2020/12/09','2020/12/10','2020/12/11','2020/12/12','2020/12/13','2020/11/18','2021/05/09','2020/11/17','2021/05/29','2020/12/14']
    fp  = ['2021/10/10','2021/08/28','2021/08/25','2021/08/11','2021/07/28','2021/06/23','2021/05/12','2021/10/16','2021/10/11','2021/09/30','2021/09/25','2021/09/11','2021/09/08','2021/08/20','2021/08/08','2021/07/23','2021/06/27','2021/01/17','2021/11/27','2021/10/07','2020/06/12','2021/06/16','2020/03/05','2021/03/11','2021/06/01','2021/06/04','2021/06/08','2021/05/10','2021/01/13','2021/01/14','2020/12/06','2020/08/31','2020/08/27','2020/08/29','2020/07/24','2020/04/19','2020/04/18','2020/05/02','2020/07/10','2020/12/17','2021/03/27','2021/03/20','2021/05/31','2021/01/05','2021/03/19','2020/12/27','2020/05/30','2020/07/23','2020/08/04','2020/11/24','2020/12/20','2021/06/07','','','','','','','','','','','','','','']
    
    correct = 0
    incorrect = 0 
    dates = np.array([])
    for i in range(times.size):
        date = times[i].strftime('%Y/%m/%d - %H:%M:%S')
        if date[0:9] == '2021/01/2' or date[0:9] == '2021/01/3' or date[0:7] == '2021/02' or date[0:7] == '2021/02' or date[0:9] == '2021/03/0' or date[0:9] == '2021/03/1':
            continue
        dates = np.append(dates, date)
    if loudness == 1:
        print('Not evaluating on data between Jan 20. 2021 - Mar 19. 2021')
    
    secG = 0
    secB = 0
    for i in dates:
        if i[0:10] in sbm:
            if loudness == 1:
                print('correct: '+i)
            correct = correct +1
            secG = secG + DT_SBM2
        else:
            incorrect = incorrect + 1 
            secB = secB + DT_SBM2
            if i[0:10] not in fp:
                if loudness == 1:
                    print('check ' + i)
            else:
                if loudness == 1:
                    print('incorrect: ' + i)
            
    print('Total correct triggers: %d' %correct)
    print('Total incorrect triggers: %d' %incorrect)
    print('----------------------------------')
    print('Correct time in SBM2 mode: %ds' %secG)
    print('Inorrect time in SBM2 mode: %ds' %secB)
    if incorrect != 0:
        print('Accuracy %0.2f' %(correct/(correct+incorrect)))

In [28]:
# Loading data
with open('SBM2_data_2021_V7.3_final.pickle', 'rb') as f:
    FP, AMP, NR, time, Epoch = pickle.load(f)
rua=1

In [17]:
# CONFIGURATION CELL #
# Confirmation default 
condef   = {'EPEAK_THRES': 200, 'FP_MIN': 5000, 'FP_MAX': 50000, 'NW_THRES': 2, 'DT_LW': 304 , 'N_LW_THRES': 5 , 'DT_SBM2':  900, 'RUA': 0.5, 'title': '_confirmation', 'type': 'default'}

# Degraded default ('_F' is removed for convenience)
degdef   = {'EPEAK_THRES': 728, 'FP_MIN': 5000, 'FP_MAX': 50000, 'NW_THRES': 3, 'DT_LW': 1200, 'N_LW_THRES': 20, 'DT_SBM2': 7200, 'RUA': 0.5, 'title': '_degraded',  'type': 'default'}

# Outdated confirmation default 
ocondef  = {'EPEAK_THRES': 728, 'FP_MIN': 5000, 'FP_MAX': 60000, 'NW_THRES': 3, 'DT_LW': 2400, 'N_LW_THRES': 20, 'DT_SBM2': 7200, 'RUA': 0.5, 'title': 'confirmation',  'type': 'outdated-default'}

# Outdated degraded default ('_F' is removed for convenience)
odegdef  = {'EPEAK_THRES': 728, 'FP_MIN': 5000, 'FP_MAX': 60000, 'NW_THRES': 3, 'DT_LW': 1200, 'N_LW_THRES': 20, 'DT_SBM2': 7200, 'RUA': 0.5, 'title': 'degraded', 'type': 'outdated-default'}

# Confirmation default HK
condefHK = {'EPEAK_THRES': 200, 'FP_MIN': 5000, 'FP_MAX': 50000, 'NW_THRES': 2, 'DT_LW': 304 , 'N_LW_THRES': 5 , 'DT_SBM2':  900, 'RUA': 0.5, 'title': '_confirmation', 'type': 'default_FROM HK LOG'}

# Degraded default ('_F' is removed for convenience) HK
degdefHK = {'EPEAK_THRES': 728, 'FP_MIN': 5000, 'FP_MAX': 50000, 'NW_THRES': 3, 'DT_LW': 1200, 'N_LW_THRES': 20, 'DT_SBM2': 7200, 'RUA': 0.5, 'title': '_degraded', 'type': 'default_FROM HK LOG'}

# Custom config 1
custom1  = {'EPEAK_THRES': 200, 'FP_MIN': 5000, 'FP_MAX': 50000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2': 7200, 'RUA': 0.5, 'title': 'custom1', 'type': 'dev'}

# Custom config 2
custom2  = {'EPEAK_THRES': 150, 'FP_MIN': 5000, 'FP_MAX': 50000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2': 7200, 'RUA': 0.5, 'title': 'custom2', 'type': 'dev'}

# Custom config 3 - Type III optimized
custom3  = {'EPEAK_THRES': 200, 'FP_MIN': 6000, 'FP_MAX': 60000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'custom3', 'type': 'dev - Type III optimized'}

# Custom config 3 - Type III optimized
custom4  = {'EPEAK_THRES': 120, 'FP_MIN': 6000, 'FP_MAX': 50000, 'NW_THRES': 1, 'DT_LW':  600, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'custom4', 'type': 'dev - Type III optimized'}

# Custom config 3 - Type III optimized
custom5  = {'EPEAK_THRES': 100, 'FP_MIN': 6000, 'FP_MAX': 50000, 'NW_THRES': 1, 'DT_LW':  600, 'N_LW_THRES': 8 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'custom5', 'type': 'dev - Type III optimized'}

# Custom config 6
custom6  = {'EPEAK_THRES':  80, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'custom6', 'type': 'dev'}

# Custom config 7
custom7  = {'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700,  'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'custom7', 'type': 'dev'}

# Flight config 1 - Flown since August 2021
flown_1  = {'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'flight_1', 'type': 'Flight - Flown since August 2021'}

In [29]:
evaluate(custom7)

Not evaluating on data between Jan 20. 2021 - Mar 19. 2021
correct: 2021/05/09 - 14:28:27
correct: 2021/05/09 - 16:28:33
correct: 2021/05/22 - 00:21:50
correct: 2021/05/22 - 07:48:30
correct: 2021/05/23 - 05:04:19
correct: 2021/05/23 - 08:56:19
correct: 2021/05/23 - 11:29:23
correct: 2021/05/29 - 00:40:47
incorrect: 2021/06/08 - 16:01:57
incorrect: 2021/06/16 - 09:33:58
correct: 2021/08/26 - 04:13:12
correct: 2021/08/26 - 18:54:29
correct: 2021/10/06 - 04:33:14
incorrect: 2021/10/07 - 15:19:50
correct: 2021/10/09 - 07:04:51
incorrect: 2021/10/16 - 07:24:52
correct: 2021/10/28 - 16:42:12
incorrect: 2021/11/27 - 00:38:46
incorrect: 2021/11/27 - 04:28:02
incorrect: 2021/11/27 - 23:55:21
correct: 2021/12/06 - 06:52:31
Total correct triggers: 14
Total incorrect triggers: 7
----------------------------------
Correct time in SBM2 mode: 100800s
Inorrect time in SBM2 mode: 50400s
Accuracy 0.67


In [30]:
evaluate(flown_1)

Not evaluating on data between Jan 20. 2021 - Mar 19. 2021
correct: 2021/05/09 - 16:13:37
correct: 2021/05/22 - 00:20:46
correct: 2021/05/22 - 07:47:26
correct: 2021/05/23 - 05:00:51
correct: 2021/05/23 - 07:44:51
correct: 2021/05/23 - 10:14:11
correct: 2021/05/29 - 00:40:15
incorrect: 2021/06/08 - 16:01:25
incorrect: 2021/06/16 - 09:33:26
correct: 2021/08/26 - 04:15:52
correct: 2021/08/26 - 18:56:21
correct: 2021/10/06 - 04:36:26
incorrect: 2021/10/07 - 15:24:22
correct: 2021/10/09 - 07:04:51
correct: 2021/10/28 - 16:41:40
incorrect: 2021/11/27 - 00:42:14
incorrect: 2021/11/27 - 04:27:30
incorrect: 2021/11/27 - 23:56:09
Total correct triggers: 12
Total incorrect triggers: 6
----------------------------------
Correct time in SBM2 mode: 86400s
Inorrect time in SBM2 mode: 43200s
Accuracy 0.67


In [16]:
print("'EPEAK_THRES':  80")
evaluate({'EPEAK_THRES':  80, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES':  90")
evaluate({'EPEAK_THRES':  90, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 100")
evaluate({'EPEAK_THRES': 100, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 110")
evaluate({'EPEAK_THRES': 110, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 120")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPbEAK_THRES': 130")
evaluate({'EPEAK_THRES': 130, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 140")
evaluate({'EPEAK_THRES': 140, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 150")
evaluate({'EPEAK_THRES': 150, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 160")
evaluate({'EPEAK_THRES': 160, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 170")
evaluate({'EPEAK_THRES': 170, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 180")
evaluate({'EPEAK_THRES': 180, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 190")
evaluate({'EPEAK_THRES': 190, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 200")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)

'EPEAK_THRES':  80
Total correct triggers: 21
Total incorrect triggers: 39
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 280800s
Accuracy 0.35
'EPEAK_THRES':  90
Total correct triggers: 21
Total incorrect triggers: 33
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 237600s
Accuracy 0.39
'EPEAK_THRES': 100
Total correct triggers: 21
Total incorrect triggers: 31
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 223200s
Accuracy 0.40
'EPEAK_THRES': 110
Total correct triggers: 21
Total incorrect triggers: 25
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 180000s
Accuracy 0.46
'EPEAK_THRES': 120
Total correct triggers: 21
Total incorrect triggers: 22
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 158400s
Accuracy 0.49
'EPEAK_THRES': 130
Total 

In [6]:
print("'DT_LW':  600")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 600, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  700")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  800")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  900")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 900, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  1000")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 1000, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  1100")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 1100, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  1200")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  1300")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 1300, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)


'DT_LW':  600
Total correct triggers: 20
Total incorrect triggers: 20
----------------------------------
Correct time in SBM2 mode: 144000s
Inorrect time in SBM2 mode: 144000s
Accuracy 0.50
'DT_LW':  700
Total correct triggers: 21
Total incorrect triggers: 21
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 151200s
Accuracy 0.50
'DT_LW':  800
Total correct triggers: 21
Total incorrect triggers: 22
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 158400s
Accuracy 0.49
'DT_LW':  900
Total correct triggers: 21
Total incorrect triggers: 22
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 158400s
Accuracy 0.49
'DT_LW':  1000
Total correct triggers: 21
Total incorrect triggers: 23
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 165600s
Accuracy 0.48
'DT_LW':  1100
Total correct triggers: 21
Total i

KeyboardInterrupt: 

In [7]:
print("'DT_LW':  700")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 1)
# GG

'DT_LW':  700
Not evaluating on data between Jan 20. 2021 - Mar 19. 2021
incorrect: 2021/01/17 - 04:54:16
incorrect: 2021/03/27 - 15:50:09
correct: 2021/05/09 - 14:27:39
correct: 2021/05/09 - 16:27:45
check 2021/05/12 - 11:26:57
correct: 2021/05/22 - 00:17:50
correct: 2021/05/22 - 07:47:26
correct: 2021/05/23 - 04:59:47
correct: 2021/05/23 - 07:43:31
correct: 2021/05/23 - 10:13:39
correct: 2021/05/29 - 00:39:43
incorrect: 2021/06/08 - 16:00:53
incorrect: 2021/06/16 - 09:33:26
check 2021/06/23 - 12:46:15
incorrect: 2021/06/27 - 05:54:16
incorrect: 2021/07/23 - 07:52:39
correct: 2021/07/24 - 01:42:31
check 2021/07/28 - 07:56:08
check 2021/08/11 - 23:44:01
correct: 2021/08/24 - 13:39:49
correct: 2021/08/24 - 17:37:57
check 2021/08/25 - 21:41:57
correct: 2021/08/26 - 04:09:12
correct: 2021/08/26 - 08:53:28
correct: 2021/08/26 - 18:53:57
check 2021/08/28 - 13:19:34
incorrect: 2021/09/08 - 23:16:03
correct: 2021/09/24 - 17:03:00
incorrect: 2021/09/25 - 18:09:08
correct: 2021/10/06 - 04:30:50

In [9]:
print("'FP_MIN': 6000")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 6000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'FP_MIN': 7000")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 7000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'FP_MIN': 8000")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 8000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'FP_MIN': 9000")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 9000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'FP_MIN': 10000")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)


'FP_MIN': 6000
Total correct triggers: 29
Total incorrect triggers: 41
----------------------------------
Correct time in SBM2 mode: 208800s
Inorrect time in SBM2 mode: 295200s
Accuracy 0.41
'FP_MIN': 7000
Total correct triggers: 25
Total incorrect triggers: 36
----------------------------------
Correct time in SBM2 mode: 180000s
Inorrect time in SBM2 mode: 259200s
Accuracy 0.41
'FP_MIN': 8000
Total correct triggers: 23
Total incorrect triggers: 28
----------------------------------
Correct time in SBM2 mode: 165600s
Inorrect time in SBM2 mode: 201600s
Accuracy 0.45
'FP_MIN': 9000
Total correct triggers: 22
Total incorrect triggers: 24
----------------------------------
Correct time in SBM2 mode: 158400s
Inorrect time in SBM2 mode: 172800s
Accuracy 0.48
'FP_MIN': 10000
Total correct triggers: 21
Total incorrect triggers: 21
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 151200s
Accuracy 0.50


In [11]:
print("'NW_THRES': 0.5")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 0.5, 'DT_LW': 700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'NW_THRES': 1")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)


'NW_THRES': 0.5
Total correct triggers: 21
Total incorrect triggers: 21
----------------------------------
Correct time in SBM2 mode: 151200s
Inorrect time in SBM2 mode: 151200s
Accuracy 0.50
'NW_THRES': 1
Total correct triggers: 15
Total incorrect triggers: 11
----------------------------------
Correct time in SBM2 mode: 108000s
Inorrect time in SBM2 mode: 79200s
Accuracy 0.58


In [17]:
print("'DT_LW':  300")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW':  300, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  400")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW':  400, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  500")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW':  500, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  600")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW':  600, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  700")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW':  700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  800")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW':  800, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW':  900")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW':  900, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW': 1000")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1000, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW': 1100")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1100, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW': 1200")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'DT_LW': 1300")
evaluate({'EPEAK_THRES': 200, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1300, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)

'DT_LW':  300
Total correct triggers: 11
Total incorrect triggers: 5
----------------------------------
Correct time in SBM2 mode: 79200s
Inorrect time in SBM2 mode: 36000s
Accuracy 0.69
'DT_LW':  400
Total correct triggers: 11
Total incorrect triggers: 6
----------------------------------
Correct time in SBM2 mode: 79200s
Inorrect time in SBM2 mode: 43200s
Accuracy 0.65
'DT_LW':  500
Total correct triggers: 11
Total incorrect triggers: 6
----------------------------------
Correct time in SBM2 mode: 79200s
Inorrect time in SBM2 mode: 43200s
Accuracy 0.65
'DT_LW':  600
Total correct triggers: 11
Total incorrect triggers: 6
----------------------------------
Correct time in SBM2 mode: 79200s
Inorrect time in SBM2 mode: 43200s
Accuracy 0.65
'DT_LW':  700
Total correct triggers: 11
Total incorrect triggers: 6
----------------------------------
Correct time in SBM2 mode: 79200s
Inorrect time in SBM2 mode: 43200s
Accuracy 0.65
'DT_LW':  800
Total correct triggers: 11
Total incorrect triggers

In [14]:
print("'N_LW_THRES': 9")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 9 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'N_LW_THRES': 8")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 8 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)


'N_LW_THRES': 9
Total correct triggers: 12
Total incorrect triggers: 6
----------------------------------
Correct time in SBM2 mode: 86400s
Inorrect time in SBM2 mode: 43200s
Accuracy 0.67
'N_LW_THRES': 8
Total correct triggers: 14
Total incorrect triggers: 6
----------------------------------
Correct time in SBM2 mode: 100800s
Inorrect time in SBM2 mode: 43200s
Accuracy 0.70


In [13]:
print("'N_LW_THRES': 7")
b
print("'N_LW_THRES': 6")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 6 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'N_LW_THRES': 5")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'N_LW_THRES': 4")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 4 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)


'N_LW_THRES': 7
Total correct triggers: 14
Total incorrect triggers: 7
----------------------------------
Correct time in SBM2 mode: 100800s
Inorrect time in SBM2 mode: 50400s
Accuracy 0.67
'N_LW_THRES': 6
Total correct triggers: 14
Total incorrect triggers: 9
----------------------------------
Correct time in SBM2 mode: 100800s
Inorrect time in SBM2 mode: 64800s
Accuracy 0.61
'N_LW_THRES': 5
Total correct triggers: 15
Total incorrect triggers: 11
----------------------------------
Correct time in SBM2 mode: 108000s
Inorrect time in SBM2 mode: 79200s
Accuracy 0.58
'N_LW_THRES': 4
Total correct triggers: 15
Total incorrect triggers: 14
----------------------------------
Correct time in SBM2 mode: 108000s
Inorrect time in SBM2 mode: 100800s
Accuracy 0.52
'N_LW_THRES': 3


KeyboardInterrupt: 

In [15]:
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 1)

Not evaluating on data between Jan 20. 2021 - Mar 19. 2021
correct: 2021/05/09 - 14:28:27
correct: 2021/05/09 - 16:28:33
correct: 2021/05/22 - 00:21:50
correct: 2021/05/22 - 07:48:30
correct: 2021/05/23 - 05:04:19
correct: 2021/05/23 - 08:56:19
correct: 2021/05/23 - 11:29:23
correct: 2021/05/29 - 00:40:47
incorrect: 2021/06/08 - 16:01:57
incorrect: 2021/06/16 - 09:33:58
correct: 2021/08/26 - 04:13:12
correct: 2021/08/26 - 18:54:29
correct: 2021/10/06 - 04:33:14
incorrect: 2021/10/07 - 15:19:50
correct: 2021/10/09 - 07:04:51
incorrect: 2021/10/16 - 07:24:52
correct: 2021/10/28 - 16:42:12
incorrect: 2021/11/27 - 00:38:46
incorrect: 2021/11/27 - 04:28:02
incorrect: 2021/11/27 - 23:55:21
correct: 2021/12/06 - 06:52:31
Total correct triggers: 14
Total incorrect triggers: 7
----------------------------------
Correct time in SBM2 mode: 100800s
Inorrect time in SBM2 mode: 50400s
Accuracy 0.67


In [16]:
print("'EPEAK_THRES': 90")
evaluate({'EPEAK_THRES': 90, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 100")
evaluate({'EPEAK_THRES': 100, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 110")
evaluate({'EPEAK_THRES': 110, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 120")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 130")
evaluate({'EPEAK_THRES': 130, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 140")
evaluate({'EPEAK_THRES': 140, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 150")
evaluate({'EPEAK_THRES': 150, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 160")
evaluate({'EPEAK_THRES': 160, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 170")
evaluate({'EPEAK_THRES': 170, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 180")
evaluate({'EPEAK_THRES': 180, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 190")
evaluate({'EPEAK_THRES': 190, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 700, 'N_LW_THRES': 7 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)


'EPEAK_THRES': 90
Total correct triggers: 16
Total incorrect triggers: 10
----------------------------------
Correct time in SBM2 mode: 115200s
Inorrect time in SBM2 mode: 72000s
Accuracy 0.62
'EPEAK_THRES': 100
Total correct triggers: 15
Total incorrect triggers: 9
----------------------------------
Correct time in SBM2 mode: 108000s
Inorrect time in SBM2 mode: 64800s
Accuracy 0.62
'EPEAK_THRES': 110
Total correct triggers: 15
Total incorrect triggers: 9
----------------------------------
Correct time in SBM2 mode: 108000s
Inorrect time in SBM2 mode: 64800s
Accuracy 0.62
'EPEAK_THRES': 120
Total correct triggers: 14
Total incorrect triggers: 7
----------------------------------
Correct time in SBM2 mode: 100800s
Inorrect time in SBM2 mode: 50400s
Accuracy 0.67
'EPEAK_THRES': 130
Total correct triggers: 13
Total incorrect triggers: 7
----------------------------------
Correct time in SBM2 mode: 93600s
Inorrect time in SBM2 mode: 50400s
Accuracy 0.65
'EPEAK_THRES': 140
Total correct tri

In [12]:
evaluate(flown_1)

Not evaluating on data between Jan 20. 2021 - Mar 19. 2021
correct: 2021/05/09 - 16:13:37
correct: 2021/05/22 - 00:20:46
correct: 2021/05/22 - 07:47:26
correct: 2021/05/23 - 05:00:51
correct: 2021/05/23 - 07:44:51
correct: 2021/05/23 - 10:14:11
correct: 2021/05/29 - 00:40:15
incorrect: 2021/06/08 - 16:01:25
incorrect: 2021/06/16 - 09:33:26
correct: 2021/08/26 - 04:15:52
correct: 2021/08/26 - 18:56:21
correct: 2021/10/06 - 04:36:26
incorrect: 2021/10/07 - 15:24:22
correct: 2021/10/09 - 07:04:51
correct: 2021/10/28 - 16:41:40
incorrect: 2021/11/27 - 00:42:14
incorrect: 2021/11/27 - 04:27:30
incorrect: 2021/11/27 - 23:56:09
Total correct triggers: 12
Total incorrect triggers: 6
----------------------------------
Correct time in SBM2 mode: 86400s
Inorrect time in SBM2 mode: 43200s
Accuracy 0.67


In [13]:
evaluate(custom6,loudness = 0)

Total correct triggers: 18
Total incorrect triggers: 21
----------------------------------
Correct time in SBM2 mode: 129600s
Inorrect time in SBM2 mode: 151200s
Accuracy 0.46


In [15]:
print("'EPEAK_THRES':  90")
evaluate({'EPEAK_THRES':  90, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 100")
evaluate({'EPEAK_THRES': 100, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 110")
evaluate({'EPEAK_THRES': 110, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 120")
evaluate({'EPEAK_THRES': 120, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 130")
evaluate({'EPEAK_THRES': 130, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 140")
evaluate({'EPEAK_THRES': 140, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 150")
evaluate({'EPEAK_THRES': 150, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 160")
evaluate({'EPEAK_THRES': 160, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 170")
evaluate({'EPEAK_THRES': 170, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 180")
evaluate({'EPEAK_THRES': 180, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)
print("'EPEAK_THRES': 190")
evaluate({'EPEAK_THRES': 190, 'FP_MIN': 10000,'FP_MAX': 90000, 'NW_THRES': 1, 'DT_LW': 1200, 'N_LW_THRES': 5 , 'DT_SBM2':  7200, 'RUA': 1.0, 'title': 'manual', 'type': 'dev'},loudness = 0)

'EPEAK_THRES':  90
Total correct triggers: 17
Total incorrect triggers: 20
----------------------------------
Correct time in SBM2 mode: 122400s
Inorrect time in SBM2 mode: 144000s
Accuracy 0.46
'EPEAK_THRES': 100
Total correct triggers: 16
Total incorrect triggers: 16
----------------------------------
Correct time in SBM2 mode: 115200s
Inorrect time in SBM2 mode: 115200s
Accuracy 0.50
'EPEAK_THRES': 110
Total correct triggers: 16
Total incorrect triggers: 14
----------------------------------
Correct time in SBM2 mode: 115200s
Inorrect time in SBM2 mode: 100800s
Accuracy 0.53
'EPEAK_THRES': 120
Total correct triggers: 16
Total incorrect triggers: 13
----------------------------------
Correct time in SBM2 mode: 115200s
Inorrect time in SBM2 mode: 93600s
Accuracy 0.55
'EPEAK_THRES': 130
Total correct triggers: 14
Total incorrect triggers: 11
----------------------------------
Correct time in SBM2 mode: 100800s
Inorrect time in SBM2 mode: 79200s
Accuracy 0.56
'EPEAK_THRES': 140
Total co

In [7]:
evaluate(flown_1)


Not evaluating on data between Jan 20. 2021 - Mar 19. 2021
check 2022/01/18 - 18:41:44
Total correct triggers: 0
Total incorrect triggers: 1
----------------------------------
Correct time in SBM2 mode: 0s
Inorrect time in SBM2 mode: 7200s
Accuracy 0.00


In [56]:
# Evaluating
evaluate(degdef)

Not evaluating on data between Jan 20. 2021 - Mar 19. 2021
Total correct triggers: 0
Total incorrect triggers: 0
----------------------------------
Correct time in SBM2 mode: 0s
Inorrect time in SBM2 mode: 0s


In [57]:
evaluate(condef)

Not evaluating on data between Jan 20. 2021 - Mar 19. 2021
incorrect: 2020/07/10 - 10:37:57
correct: 2020/11/17 - 13:58:20
incorrect: 2020/12/27 - 04:45:51
incorrect: 2020/12/27 - 05:21:35
incorrect: 2020/12/27 - 07:13:23
incorrect: 2020/12/27 - 07:37:39
incorrect: 2020/12/27 - 08:15:15
incorrect: 2020/12/27 - 08:33:23
incorrect: 2020/12/27 - 09:01:23
correct: 2021/05/09 - 16:13:37
correct: 2021/05/22 - 00:41:02
correct: 2021/05/22 - 07:31:42
correct: 2021/05/22 - 07:46:54
correct: 2021/05/23 - 08:56:19
correct: 2021/05/23 - 10:14:27
correct: 2021/05/23 - 11:28:51
correct: 2021/05/23 - 11:44:03
correct: 2021/05/29 - 00:41:03
incorrect: 2021/05/31 - 19:38:56
incorrect: 2021/05/31 - 20:01:04
incorrect: 2021/06/08 - 16:01:57
incorrect: 2021/06/16 - 09:33:26
incorrect: 2021/06/16 - 09:48:38
incorrect: 2021/06/16 - 10:03:50
incorrect: 2021/06/16 - 10:22:30
correct: 2021/08/26 - 18:56:37
correct: 2021/08/26 - 19:11:49
Total correct triggers: 12
Total incorrect triggers: 15
------------------

In [58]:
evaluate(custom1, loudness=0)

Total correct triggers: 13
Total incorrect triggers: 11
----------------------------------
Correct time in SBM2 mode: 93600s
Inorrect time in SBM2 mode: 79200s
Accuracy 0.54


In [59]:
evaluate(custom2, loudness=0)

Total correct triggers: 17
Total incorrect triggers: 25
----------------------------------
Correct time in SBM2 mode: 122400s
Inorrect time in SBM2 mode: 180000s
Accuracy 0.40


In [65]:
evaluate(custom3, loudness=0)

Total correct triggers: 14
Total incorrect triggers: 8
----------------------------------
Correct time in SBM2 mode: 100800s
Inorrect time in SBM2 mode: 57600s
Accuracy 0.64


In [61]:
evaluate(custom4, loudness=0)

Total correct triggers: 15
Total incorrect triggers: 17
----------------------------------
Correct time in SBM2 mode: 108000s
Inorrect time in SBM2 mode: 122400s
Accuracy 0.47


In [62]:
evaluate(custom5, loudness=0)

Total correct triggers: 14
Total incorrect triggers: 20
----------------------------------
Correct time in SBM2 mode: 100800s
Inorrect time in SBM2 mode: 144000s
Accuracy 0.41


In [63]:
evaluate(custom3, loudness=1)

Not evaluating on data between Jan 20. 2021 - Mar 19. 2021
incorrect: 2020/05/30 - 18:08:19
incorrect: 2020/06/12 - 15:09:09
correct: 2020/11/17 - 13:58:20
correct: 2020/11/18 - 19:39:56
correct: 2020/12/13 - 11:05:33
incorrect: 2020/12/27 - 04:45:51
incorrect: 2020/12/27 - 07:14:11
incorrect: 2020/12/27 - 09:14:11
correct: 2021/05/09 - 16:13:37
correct: 2021/05/22 - 00:20:46
correct: 2021/05/22 - 04:10:38
correct: 2021/05/22 - 07:30:38
correct: 2021/05/22 - 09:30:38
correct: 2021/05/23 - 05:00:51
correct: 2021/05/23 - 07:44:51
correct: 2021/05/23 - 10:14:11
correct: 2021/05/29 - 00:40:15
incorrect: 2021/05/31 - 19:40:16
incorrect: 2021/06/08 - 16:01:25
incorrect: 2021/06/16 - 09:33:26
correct: 2021/08/26 - 04:15:52
correct: 2021/08/26 - 18:56:21
Total correct triggers: 14
Total incorrect triggers: 8
----------------------------------
Correct time in SBM2 mode: 100800s
Inorrect time in SBM2 mode: 57600s
Accuracy 0.64


In [64]:
evaluate(custom4, loudness=1)

Not evaluating on data between Jan 20. 2021 - Mar 19. 2021
incorrect: 2020/03/05 - 08:13:08
incorrect: 2020/05/02 - 09:50:02
incorrect: 2020/05/30 - 18:03:47
incorrect: 2020/06/12 - 15:09:09
incorrect: 2020/07/10 - 03:54:45
incorrect: 2020/07/10 - 10:36:21
check 2020/07/27 - 22:58:16
incorrect: 2020/08/04 - 20:06:54
correct: 2020/11/17 - 13:58:52
incorrect: 2020/11/24 - 15:17:05
correct: 2020/12/10 - 08:11:39
correct: 2020/12/13 - 10:48:29
incorrect: 2020/12/27 - 04:44:31
incorrect: 2020/12/27 - 07:13:39
incorrect: 2020/12/27 - 09:13:39
correct: 2021/05/09 - 14:28:27
correct: 2021/05/09 - 16:28:33
correct: 2021/05/22 - 00:21:50
correct: 2021/05/22 - 04:15:10
correct: 2021/05/22 - 07:31:26
correct: 2021/05/23 - 05:04:19
correct: 2021/05/23 - 08:56:19
correct: 2021/05/23 - 11:29:23
correct: 2021/05/29 - 00:40:47
incorrect: 2021/05/31 - 19:40:48
incorrect: 2021/06/08 - 16:01:57
incorrect: 2021/06/16 - 09:33:58
check 2021/08/09 - 01:48:34
check 2021/08/19 - 17:42:45
correct: 2021/08/26 - 0